In [33]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [6]:
#Load the data

df = pd.read_csv('Company_Data (1).csv')
df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


## EDA

In [7]:
df.describe()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,7.496325,124.975000,68.657500,6.635000,264.840000,115.795000,53.322500,13.900000
std,2.824115,15.334512,27.986037,6.650364,147.376436,23.676664,16.200297,2.620528
min,0.000000,77.000000,21.000000,0.000000,10.000000,24.000000,25.000000,10.000000
25%,5.390000,115.000000,42.750000,0.000000,139.000000,100.000000,39.750000,12.000000
50%,7.490000,125.000000,69.000000,5.000000,272.000000,117.000000,54.500000,14.000000
75%,9.320000,135.000000,91.000000,12.000000,398.500000,131.000000,66.000000,16.000000
max,16.270000,175.000000,120.000000,29.000000,509.000000,191.000000,80.000000,18.000000


In [8]:
df.shape

(400, 11)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


In [10]:
df[df.duplicated()]

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US


In [11]:
df.duplicated().sum()

0

## Encoding

In [12]:
import category_encoders as ce

In [13]:
Ordinal = ce.OrdinalEncoder(cols = ['ShelveLoc','Urban','US'])
Ordinal

OrdinalEncoder(cols=['ShelveLoc', 'Urban', 'US'])

In [14]:
df1 = Ordinal.fit_transform(df)

In [15]:
df1.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,1,42,17,1,1
1,11.22,111,48,16,260,83,2,65,10,1,1
2,10.06,113,35,10,269,80,3,59,12,1,1
3,7.40,117,100,4,466,97,3,55,14,1,1
4,4.15,141,64,3,340,128,1,38,13,1,2


In [16]:
Target_var_sale = []
for value in df1["Sales"]:
    if value<= 8:
        Target_var_sale.append("low")
    else:
        Target_var_sale.append("high")

df1["Target_var_sale"]= Target_var_sale

In [17]:
df1.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,Target_var_sale
0,9.50,138,73,11,276,120,1,42,17,1,1,high
1,11.22,111,48,16,260,83,2,65,10,1,1,high
2,10.06,113,35,10,269,80,3,59,12,1,1,high
3,7.40,117,100,4,466,97,3,55,14,1,1,low
4,4.15,141,64,3,340,128,1,38,13,1,2,low


In [18]:
df1 = df1.drop(columns= 'Sales')

In [19]:
df1.head()

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,Target_var_sale
0,138,73,11,276,120,1,42,17,1,1,high
1,111,48,16,260,83,2,65,10,1,1,high
2,113,35,10,269,80,3,59,12,1,1,high
3,117,100,4,466,97,3,55,14,1,1,low
4,141,64,3,340,128,1,38,13,1,2,low


In [20]:
x = df1.iloc[:,:-1]
y = df1['Target_var_sale']

## Splitting the data

In [21]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=1)

### Individual Models

In [23]:
lg = LogisticRegression()
lg.fit(xtrain,ytrain)
ypred = lg.predict(xtest)

print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

        high       0.79      0.72      0.75        32
         low       0.82      0.88      0.85        48

    accuracy                           0.81        80
   macro avg       0.81      0.80      0.80        80
weighted avg       0.81      0.81      0.81        80



In [24]:
print('Training accuracy:', lg.score(xtrain,ytrain))
print('Testing Accuracy:', lg.score(xtest,ytest))

Training accuracy: 0.83125
Testing Accuracy: 0.8125


In [25]:
#prediction function
def predict(model):
    model = model.fit(xtrain,ytrain)
    ypred = model.predict(xtest)
    
    print(classification_report(ytest,ypred))
    print(model.score(xtrain,ytrain))
    print(model.score(xtest,ytest))

In [26]:
predict(LogisticRegression())

              precision    recall  f1-score   support

        high       0.79      0.72      0.75        32
         low       0.82      0.88      0.85        48

    accuracy                           0.81        80
   macro avg       0.81      0.80      0.80        80
weighted avg       0.81      0.81      0.81        80

0.83125
0.8125


In [27]:
#Decision Tree, Random forest, KNN

In [28]:
predict(DecisionTreeClassifier())

              precision    recall  f1-score   support

        high       0.63      0.75      0.69        32
         low       0.81      0.71      0.76        48

    accuracy                           0.73        80
   macro avg       0.72      0.73      0.72        80
weighted avg       0.74      0.72      0.73        80

1.0
0.725


In [29]:
predict(KNeighborsClassifier())

              precision    recall  f1-score   support

        high       0.52      0.44      0.47        32
         low       0.66      0.73      0.69        48

    accuracy                           0.61        80
   macro avg       0.59      0.58      0.58        80
weighted avg       0.60      0.61      0.61        80

0.76875
0.6125


In [30]:
predict(BaggingClassifier(DecisionTreeClassifier()))

              precision    recall  f1-score   support

        high       0.65      0.88      0.75        32
         low       0.89      0.69      0.78        48

    accuracy                           0.76        80
   macro avg       0.77      0.78      0.76        80
weighted avg       0.80      0.76      0.76        80

0.9875
0.7625


In [31]:
predict(BaggingClassifier(KNeighborsClassifier()))

              precision    recall  f1-score   support

        high       0.48      0.44      0.46        32
         low       0.65      0.69      0.67        48

    accuracy                           0.59        80
   macro avg       0.56      0.56      0.56        80
weighted avg       0.58      0.59      0.58        80

0.75625
0.5875


## Random Forest

In [32]:
predict(RandomForestClassifier())

              precision    recall  f1-score   support

        high       0.82      0.72      0.77        32
         low       0.83      0.90      0.86        48

    accuracy                           0.82        80
   macro avg       0.82      0.81      0.81        80
weighted avg       0.82      0.82      0.82        80

1.0
0.825
